#Bài tập thực hành - Lập trình máy học cho python - CS116.M11



MSSV: 19521299

Tên: Nguyễn Chí Cường

Ngày: 23/11/2021

Tuần 12

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, ShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [38]:
df_loan = pd.read_csv('loan_data.csv')
df_loan

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [39]:
#xử lý đơn sơ
df_loan = df_loan.dropna(how='any',axis=0)
label = df_loan['Loan_Status'] #lấy label
df_loan = df_loan.drop(['Loan_ID', 'Loan_Status'], axis=1)
df_loan = df_loan.replace({'Dependents': {'3+': 3}})
df_loan

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural
610,Male,Yes,3,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban


In [40]:
#encoding data
df_onehot = df_loan.copy()
columns=['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']
df_onehot = pd.get_dummies(df_onehot, columns=columns)

In [41]:
#scale data
scaler = StandardScaler()
data_scale = scaler.fit_transform(df_onehot)

In [42]:
#cross validation
def Kflod_score(model, n):
  cv = ShuffleSplit(n_splits=n, test_size=0.2, random_state=0)
  score = cross_val_score(model, data_scale, label, cv=cv)
  return score

#1. Gradient Boosting

In [43]:
from sklearn.ensemble import GradientBoostingClassifier as GBM

parameter_gb = {
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    }
clf_gbm = GBM(n_estimators=10, learning_rate=1.0, max_depth=1, random_state=0, loss="deviance",
              subsample=0.5)
tuning_gb = GridSearchCV(clf_gbm, parameter_gb, refit = True, cv = 5)
tuning_gb.fit(data_scale, label)

GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(learning_rate=1.0,
                                                  max_depth=1, n_estimators=10,
                                                  random_state=0,
                                                  subsample=0.5),
             param_grid={'criterion': ['friedman_mse', 'mae'],
                         'max_features': ['log2', 'sqrt'],
                         'min_samples_leaf': array([0.1       , 0.13636364, 0.17272727, 0.20909091, 0.24545455,
       0.28181818, 0.31818182, 0.35454545, 0.39090909, 0.42727273,
       0.46363636, 0.5       ]),
                         'min_samples_split': array([0.1       , 0.13636364, 0.17272727, 0.20909091, 0.24545455,
       0.28181818, 0.31818182, 0.35454545, 0.39090909, 0.42727273,
       0.46363636, 0.5       ])})

In [44]:
print("best model:", tuning_gb.best_estimator_)
print("best parameter:", tuning_gb.best_params_)
print("best accuracy:", tuning_gb.best_score_)

best model: GradientBoostingClassifier(learning_rate=1.0, max_depth=1, max_features='log2',
                           min_samples_leaf=0.2090909090909091,
                           min_samples_split=0.1, n_estimators=10,
                           random_state=0, subsample=0.5)
best parameter: {'criterion': 'friedman_mse', 'max_features': 'log2', 'min_samples_leaf': 0.2090909090909091, 'min_samples_split': 0.1}
best accuracy: 0.6916666666666667


In [45]:
#cross validation
score_gb = Kflod_score(tuning_gb.best_estimator_, 5)
score_gb

array([0.63541667, 0.67708333, 0.69791667, 0.64583333, 0.65625   ])

#2. XGBoost

In [46]:
from xgboost import XGBClassifier
clf_xgb = XGBClassifier()
parameters_xgb = {
        'num_boost_round': [100, 250, 500],
        'eta': [0.05, 0.1, 0.3],
        'max_depth': [6, 9, 12],
        'subsample': [0.9, 1.0],
        'colsample_bytree': [0.9, 1.0],
    }
tuning_xgb = GridSearchCV(clf_xgb, parameters_xgb, refit = True, cv = 5)
tuning_xgb.fit(data_scale, label)

GridSearchCV(cv=5, estimator=XGBClassifier(),
             param_grid={'colsample_bytree': [0.9, 1.0],
                         'eta': [0.05, 0.1, 0.3], 'max_depth': [6, 9, 12],
                         'num_boost_round': [100, 250, 500],
                         'subsample': [0.9, 1.0]})

In [47]:
print("best model:", tuning_xgb.best_estimator_)
print("best parameter:", tuning_xgb.best_params_)
print("best accuracy:", tuning_xgb.best_score_)

best model: XGBClassifier(colsample_bytree=0.9, eta=0.05, max_depth=9, num_boost_round=100,
              subsample=0.9)
best parameter: {'colsample_bytree': 0.9, 'eta': 0.05, 'max_depth': 9, 'num_boost_round': 100, 'subsample': 0.9}
best accuracy: 0.7895833333333334


In [48]:
score_xgb = Kflod_score(tuning_xgb.best_estimator_, 5)
score_xgb

array([0.6875    , 0.79166667, 0.8125    , 0.73958333, 0.76041667])

#Summary

In [49]:
print("Accuracy with Gradient Boosting:", score_gb.mean())
print("Accuracy with XGBoost:", score_xgb.mean())

Accuracy with Gradient Boosting: 0.6625
Accuracy with XGBoost: 0.7583333333333333


#End